### ライブラリ・データセットのインポートと前処理関数

In [3]:
import glob
from gensim import corpora, models

In [4]:
docs=[]


for x in glob.glob("text/*/*.txt"):
    text = ""
    with open(x) as f:
        next(f)
        next(f)
        for line in f:
            text = text + line
        f.close()
        docs.append((x.split("/")[1],text))
len(docs)
# #文書の数は7376ある。

7376

In [5]:
import re
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter, LowerCaseFilter, ExtractAttributeFilter

def clean_text(text):
    # 数字を削除する
    text = re.sub(r'\d+', '', text)
    
    # 特殊文字や記号を削除する
    text = re.sub(r'[^\w\s]', '', text)
    
    
    # 見出しのHTTPの部分を削除する
    text = re.sub(r'httpnewslivedoorcomarticledetail', '', text)
    return text


def preprocess_text(text):
    # テキストのクリーニング
    text = clean_text(text)

    #形態素解析とトークン化
    tokenizer = Tokenizer()
    token_filters = [
        POSKeepFilter(['名詞', '動詞', '形容詞']),  # 名詞、動詞、形容詞のみ保持する
        LowerCaseFilter(),  # 全ての単語を小文字に変換する
        ExtractAttributeFilter('base_form')  # 単語の基本形に変換する
    ]
    analyzer = Analyzer(tokenizer=tokenizer, token_filters=token_filters)
    tokens = [token for token in analyzer.analyze(text)]
    
    # ストップワードの除去
    stop_words = [
    'あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた','ある', 'あれ', 'いくつ', 'いつ','いる',
    'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち','しまう',
    'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ',
    'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ','する',
    'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ',
    'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん','できる', 'とおり',
    'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば',
    'なに', 'など','なる', 'なん', 'の','はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ',
    'へん', 'ぺん', 'ほう', 'ほか','れる','られる', ...]
    stop_words_set = set(stop_words)

    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    return filtered_tokens

In [6]:
docs = []

for file_path in glob.glob("text/*/*.txt"):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        preprocessed_text = preprocess_text(text)
        docs.append((file_path, preprocessed_text))

# テキストの前処理結果を表示
# for doc in docs:
#     file_path, preprocessed_text = doc
#     print("File:", file_path)
#     print("Preprocessed Text:", preprocessed_text)
#     print()

## LDA準備

In [7]:
import gensim
from gensim import corpora

#テキストを数値化するための辞書を作成
dictionary = corpora.Dictionary([doc[1] for doc in docs])

#テキストをベクトル化
corpus = [dictionary.doc2bow(doc[1]) for doc in docs]

#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される
print(dictionary.token2id)

{'amazlet': 0, 'amazoncojp': 1, 'at': 2, 'dear': 3, 'dvd': 4, 'dvdposted': 5, 't': 6, 'with': 7, 'あきらめる': 8, 'える': 9, 'きい': 10, 'くる': 11, 'くれる': 12, 'その後': 13, 'つける': 14, 'はじめる': 15, 'ひとり': 16, 'まま': 17, 'れい': 18, 'アミューズソフトエンタテインメント': 19, 'エンター': 20, 'サスペンス': 21, 'シングル': 22, 'テレビ': 23, 'デビュー': 24, 'ドラマ': 25, 'ブルー': 26, 'ライター': 27, 'ランキング': 28, 'ルポ': 29, 'レイ': 30, 'ヶ月': 31, '一': 32, '一生': 33, '不倫': 34, '世界': 35, '世間': 36, '両親': 37, '中傷': 38, '中島': 39, '中絶': 40, '主題歌': 41, '乃': 42, '二': 43, '井上': 44, '人': 45, '人目': 46, '休止': 47, '作': 48, '作品': 49, '作家': 50, '俗説': 51, '信じる': 52, '光代': 53, '八': 54, '公開': 55, '再会': 56, '出る': 57, '出演': 58, '出産': 59, '初': 60, '別れ': 61, '別れる': 62, '劇団': 63, '劇場': 64, '化': 65, '北': 66, '千草': 67, '半年': 68, '博美': 69, '原作': 70, '受ける': 71, '各地': 72, '名付ける': 73, '名前': 74, '向き合う': 75, '向ける': 76, '和子': 77, '嘉': 78, '地': 79, '地上': 80, '地下': 81, '埋める': 82, '場所': 83, '売り上げ': 84, '変える': 85, '大半': 86, '大学生': 87, '女': 88, '妻': 89, '妻子': 90, '娘': 91, '子': 92, '子供': 93, '孝史':

In [8]:
#corpusにはdocumentごとに単語の(ID、出現回数)のリストが得られる

print(corpus[:][:10]) #文章での出現順でなく辞書IDの若い順なことに注意

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 4), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 3), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 7), (55, 1), (56, 1), (57, 2), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 2), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 6), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 5), (89, 3), (90, 1), (91, 1), (92, 4), (93, 4), (94, 1), (95, 2), (96, 1), (97, 1), (98, 2), (99, 2), (100, 3), (101, 1), (102, 6), (103, 1), (104, 3), (105, 4), (106, 3), (107, 4), (108, 1), (109, 1), (110, 1)

## LDA学習

In [9]:
# #(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 
# ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            num_topics=10,
#                                            id2word=dictionary,
#                                            alpha=0.1,                             #optional LDAのハイパーパラメータalpha
#                                            eta=0.1,                                 #optional LDAのハイパーパラメータbeta
#                                            #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
#                                           )

# topics = ldamodel.print_topics(num_words=15)
# for topic in topics:
#     print(topic)


#LDA学習
num_topics = 10  # トピックの数を指定
ldamodel = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

## パラメータの確認

In [10]:
# トピックの表示
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.013*"対応" + 0.012*"スマート" + 0.011*"フォン" + 0.011*"搭載" + 0.011*"ドコモ" + 0.010*"発売" + 0.010*"モデル" + 0.009*"発表" + 0.009*"smax" + 0.008*"android"
Topic: 1 
Words: 0.013*"自分" + 0.013*"人" + 0.009*"的" + 0.009*"よう" + 0.008*"仕事" + 0.007*"彼" + 0.007*"転職" + 0.006*"いく" + 0.006*"占い" + 0.006*"もの"
Topic: 2 
Words: 0.018*"ん" + 0.017*"人" + 0.014*"思う" + 0.010*"歳" + 0.009*"ない" + 0.009*"女性" + 0.009*"女" + 0.009*"言う" + 0.009*"よう" + 0.009*"結婚"
Topic: 3 
Words: 0.017*"日本" + 0.014*"日" + 0.013*"氏" + 0.013*"韓国" + 0.010*"選手" + 0.009*"代表" + 0.007*"t" + 0.007*"試合" + 0.007*"戦" + 0.006*"サッカー"
Topic: 4 
Words: 0.020*"映画" + 0.008*"公開" + 0.007*"作品" + 0.007*"作" + 0.006*"月日" + 0.005*"人" + 0.005*"監督" + 0.005*"年" + 0.005*"世界" + 0.005*"本"
Topic: 5 
Words: 0.012*"話題" + 0.008*"記事" + 0.008*"ネット" + 0.008*"円" + 0.008*"関連" + 0.007*"万" + 0.007*"情報" + 0.006*"人" + 0.006*"者" + 0.006*"t"
Topic: 6 
Words: 0.006*"みる" + 0.006*"よう" + 0.006*"さ" + 0.005*"使う" + 0.005*"もの" + 0.005*"写真" + 0.005*"肌" + 0.005*"円" + 0.005*"中" + 0.004

In [33]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの
for n,item in enumerate(corpus[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(1, 0.19572616), (2, 0.20801465), (4, 0.59426296)]
document ID 1:[(1, 0.131295), (2, 0.16692498), (3, 0.028116774), (4, 0.62020874), (5, 0.044891004)]
document ID 2:[(1, 0.108272426), (2, 0.037479635), (4, 0.83708215), (9, 0.016046071)]
document ID 3:[(2, 0.32494754), (4, 0.41497388), (6, 0.23953575), (9, 0.017845744)]
document ID 4:[(2, 0.2257187), (3, 0.23293531), (4, 0.52641404), (9, 0.013467499)]
document ID 5:[(1, 0.25463334), (2, 0.5441712), (4, 0.18171726), (6, 0.015026058)]
document ID 6:[(2, 0.302895), (4, 0.6928192)]
document ID 7:[(0, 0.03632125), (2, 0.0971219), (4, 0.76675993), (7, 0.014126678), (8, 0.023939839), (9, 0.059931234)]
document ID 8:[(1, 0.03304311), (2, 0.07245333), (4, 0.7715287), (6, 0.12184654)]
document ID 9:[(3, 0.3259038), (4, 0.6709306)]


In [37]:
topics = ldamodel.print_topics(num_topics=9, num_words=10)
sorted_topics = sorted(topics, key=lambda x: int(x[0]))
for topic in sorted_topics:
    print(topic)
# #documentのcategory
# categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

(1, '0.013*"自分" + 0.013*"人" + 0.009*"的" + 0.009*"よう" + 0.008*"仕事" + 0.007*"彼" + 0.007*"転職" + 0.006*"いく" + 0.006*"占い" + 0.006*"もの"')
(2, '0.018*"ん" + 0.017*"人" + 0.014*"思う" + 0.010*"歳" + 0.009*"ない" + 0.009*"女性" + 0.009*"女" + 0.009*"言う" + 0.009*"よう" + 0.009*"結婚"')
(3, '0.017*"日本" + 0.014*"日" + 0.013*"氏" + 0.013*"韓国" + 0.010*"選手" + 0.009*"代表" + 0.007*"t" + 0.007*"試合" + 0.007*"戦" + 0.006*"サッカー"')
(4, '0.020*"映画" + 0.008*"公開" + 0.007*"作品" + 0.007*"作" + 0.006*"月日" + 0.005*"人" + 0.005*"監督" + 0.005*"年" + 0.005*"世界" + 0.005*"本"')
(5, '0.012*"話題" + 0.008*"記事" + 0.008*"ネット" + 0.008*"円" + 0.008*"関連" + 0.007*"万" + 0.007*"情報" + 0.006*"人" + 0.006*"者" + 0.006*"t"')
(6, '0.006*"みる" + 0.006*"よう" + 0.006*"さ" + 0.005*"使う" + 0.005*"もの" + 0.005*"写真" + 0.005*"肌" + 0.005*"円" + 0.005*"中" + 0.004*"女性"')
(7, '0.042*"更新" + 0.030*"ソフトウェア" + 0.014*"場合" + 0.013*"くださる" + 0.013*"表示" + 0.012*"利用" + 0.012*"中" + 0.012*"設定" + 0.011*"画面" + 0.010*"ソフトバンク"')
(8, '0.031*"アプリ" + 0.017*"android" + 0.011*"smax" + 0.011*"iphone" 

In [38]:
from nltk.corpus import reuters

# reutersコーパス内のファイル識別子を取得
fileids = reuters.fileids()

# 各ファイルのカテゴリーを取得
categories = [reuters.categories(fileid) for fileid in fileids]


## 可視化

In [39]:
#import pyLDAvis.gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [40]:
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [41]:
#保存する
pyLDAvis.save_html(lda_display,'vis.html')

In [42]:
!ls

Assignment0516.ipynb     NLTK_Kmeans_sample.ipynb ja.tsv
Assignment0523 .ipynb    README.md                jawiki.doc2vec.dbow300d
Clustering.ipynb         Similarity.ipynb         ldcc-20140209.tar.gz
LDA_EN.ipynb             ja.bin                   nlp_country.csv
LDA_TA.ipynb             ja.bin.syn0.npy          text
May21 news.csv           ja.bin.syn1neg.npy       vis.html


In [44]:
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# トピック数を変化させる
for num_topics in range(2, 18):
    # モデルの構築
    ldamodel = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
    
    # トピック表示
    print("Number of Topics:", num_topics)
    for idx, topic in ldamodel.print_topics(-1):
        print('Topic: {} \nWords: {}'.format(idx, topic))
    print("-----------------------")


Number of Topics: 2
Topic: 0 
Words: 0.009*"人" + 0.007*"ん" + 0.006*"思う" + 0.006*"よう" + 0.005*"女性" + 0.005*"ない" + 0.005*"自分" + 0.005*"歳" + 0.004*"女" + 0.004*"結婚"
Topic: 1 
Words: 0.007*"更新" + 0.007*"android" + 0.007*"smax" + 0.006*"対応" + 0.006*"アプリ" + 0.006*"スマート" + 0.006*"フォン" + 0.005*"ソフトウェア" + 0.005*"ドコモ" + 0.005*"エスマックス"
-----------------------
Number of Topics: 3
Topic: 0 
Words: 0.005*"搭載" + 0.005*"画面" + 0.005*"対応" + 0.004*"人" + 0.004*"よう" + 0.004*"発売" + 0.004*"モデル" + 0.004*"機能" + 0.004*"設定" + 0.004*"発表"
Topic: 1 
Words: 0.007*"人" + 0.007*"ん" + 0.006*"よう" + 0.006*"歳" + 0.005*"ない" + 0.005*"思う" + 0.005*"自分" + 0.005*"女性" + 0.004*"言う" + 0.004*"結婚"
Topic: 2 
Words: 0.007*"人" + 0.006*"android" + 0.005*"smax" + 0.005*"更新" + 0.005*"アプリ" + 0.005*"よう" + 0.005*"スマート" + 0.005*"思う" + 0.005*"フォン" + 0.004*"ドコモ"
-----------------------
Number of Topics: 4
Topic: 0 
Words: 0.010*"人" + 0.009*"ん" + 0.007*"思う" + 0.007*"よう" + 0.006*"歳" + 0.006*"女性" + 0.006*"ない" + 0.006*"自分" + 0.006*"女" + 0.006*"結婚"
To

Number of Topics: 11
Topic: 0 
Words: 0.010*"smaxjp" + 0.009*"対応" + 0.009*"エスマックス" + 0.008*"搭載" + 0.007*"利用" + 0.006*"向け" + 0.006*"htc" + 0.005*"モデル" + 0.005*"サービス" + 0.005*"発表"
Topic: 1 
Words: 0.022*"映画" + 0.006*"賞" + 0.004*"作" + 0.004*"公開" + 0.004*"作品" + 0.004*"観る" + 0.004*"人" + 0.004*"年" + 0.003*"月日" + 0.003*"登場"
Topic: 2 
Words: 0.034*"アプリ" + 0.012*"android" + 0.008*"play" + 0.007*"peachy" + 0.006*"ld" + 0.006*"名" + 0.006*"紹介" + 0.005*"iphone" + 0.005*"google" + 0.005*"みる"
Topic: 3 
Words: 0.014*"smax" + 0.010*"フォン" + 0.010*"スマート" + 0.009*"エスマックス" + 0.009*"円" + 0.007*"android" + 0.007*"発売" + 0.007*"ntt" + 0.006*"搭載" + 0.006*"モデル"
Topic: 4 
Words: 0.006*"思う" + 0.006*"ん" + 0.006*"よう" + 0.006*"店" + 0.005*"人" + 0.005*"さ" + 0.004*"自転車" + 0.004*"みる" + 0.004*"日" + 0.004*"ない"
Topic: 5 
Words: 0.009*"日本" + 0.008*"日" + 0.005*"年" + 0.005*"t" + 0.005*"人" + 0.004*"者" + 0.004*"よう" + 0.004*"中" + 0.003*"選手" + 0.003*"世界"
Topic: 6 
Words: 0.018*"ソフトバンク" + 0.007*"人" + 0.006*"氏" + 0.005*"ん" + 0.005*"

Number of Topics: 16
Topic: 0 
Words: 0.020*"円" + 0.008*"位" + 0.007*"韓国" + 0.006*"万" + 0.006*"クリスマス" + 0.006*"日本" + 0.005*"話題" + 0.005*"月々" + 0.005*"人" + 0.004*"女性"
Topic: 1 
Words: 0.013*"人" + 0.013*"ん" + 0.009*"思う" + 0.008*"結婚" + 0.008*"女" + 0.008*"よう" + 0.007*"女性" + 0.007*"歳" + 0.006*"ない" + 0.006*"私"
Topic: 2 
Words: 0.006*"円" + 0.006*"モデル" + 0.006*"店" + 0.004*"年" + 0.004*"的" + 0.004*"よう" + 0.004*"発売" + 0.004*"価格" + 0.004*"日" + 0.004*"t"
Topic: 3 
Words: 0.009*"カメラ" + 0.008*"撮影" + 0.007*"画素" + 0.006*"機能" + 0.006*"写真" + 0.006*"色" + 0.006*"搭載" + 0.006*"肌" + 0.005*"使う" + 0.005*"発売"
Topic: 4 
Words: 0.011*"人" + 0.007*"よう" + 0.006*"みる" + 0.006*"思う" + 0.005*"ん" + 0.005*"紹介" + 0.005*"くれる" + 0.005*"ない" + 0.005*"多い" + 0.004*"中"
Topic: 5 
Words: 0.006*"optimus" + 0.006*"記事" + 0.006*"思う" + 0.006*"shd" + 0.005*"phone" + 0.005*"よう" + 0.005*"ん" + 0.005*"人" + 0.005*"女性" + 0.005*"ドコモ"
Topic: 6 
Words: 0.033*"独" + 0.016*"婚" + 0.015*"活" + 0.008*"人" + 0.007*"女" + 0.006*"結婚" + 0.005*"よう" + 0.004*"思う" +

/Users/junyaatake/miniconda3/lib/python3.7/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/junyaatake/miniconda3/lib/python3.7/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/Users/junyaatake/miniconda3/lib/python3.7/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/Users/junyaatake/miniconda3/lib/python3.7/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
